<a href="https://colab.research.google.com/github/haosulab/SAPIEN-tutorial/blob/master/basics/3_physics.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Note: Some core features of SAPIEN are not available on Colab, including the interactive viewer and ray-tracing functionalities. You need to run SAPIEN locally for full features. A more comprehensive tutorial including features that are only available to local environment can be found at [SAPIEN's documentation](https://sapien.ucsd.edu/docs/latest/index.html).

# Basics Tutorial 3: Physics

Since SAPIEN is a physical simulation framework, we would like to showcase how to change physical properties which lead to different behaviors.

In this tutorial, you will learn the following:

- Use `SceneConfig` to initialize default physical properties
- Use `PhysicalMaterial` to set different physical materials
- Create kinematic actors
- Enable damping for actors
- Get kinematic quantities (pose, velocity, angular velocity) of an actor

## Preparation

**Note: you need GPU runtime to run the notebook.**

In [ ]:
%pip install sapien matplotlib

import sapien.core as sapien
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from  transforms3d.quaternions import axangle2quat

%matplotlib inline

The example illustrates an object sliding down the slope. You can run the script with different arguments.

In [ ]:
# Hyperparameters
obj = 'box' # The type of object. Choices are 'box' and 'sphere'
gravity = -9.8 # Z-axis gravity
angle = 30.0 # The angle of the slope
offset = 0.1 # Z-offset of the slope above the ground
static_friction = 0.3 # Static friction
dynamic_friction = 0.3 # Dynamic Friction
restitution = 0.1 # Restitution (elasticity of collision)
linear_damping = 0.0 # Linear damping (resistance proportional to linear velocity)
angular_damping = 0.0 # Angular damping (resistance proportional to angular velocity)


## Set default physical properties

Default physical properties can be specified when a scene is created. Those properties include gravity, static and dynamic friction, as well as [restitution](https://en.wikipedia.org/wiki/Coefficient_of_restitution) (elasticity of collision).

In [ ]:
engine = sapien.Engine()
renderer = sapien.SapienRenderer()
engine.set_renderer(renderer)

scene_config = sapien.SceneConfig() # The default physical properties can be modified through sapien.SceneConfig
print(scene_config.gravity)
print(scene_config.default_static_friction)
print(scene_config.default_dynamic_friction)
print(scene_config.default_restitution)
scene_config.gravity = np.array([0.0, 0.0, gravity])
scene = engine.create_scene(scene_config)
scene.set_timestep(1 / 100.0)

`SceneConfig` describes default physical properties, and can be passed to `Scene`.

## Set physical materials

`PhysicalMaterial` describes physical (contact) properties (friction and restitution) of the material of an actor. It can be specified when an actor is created. If not provided, the default physical material, induced by the scene’s default physical properties, will be used. Note that `PhysicalMaterial` can only be created by `create_physical_material(...)`.

In [ ]:
physical_material: sapien.PhysicalMaterial = scene.create_physical_material(
    static_friction=static_friction,
    dynamic_friction=dynamic_friction,
    restitution=restitution,
)

Some other physical properties, like density, are directly provided to collision shapes. 

In [ ]:
def create_sphere(
        scene: sapien.Scene,
        pose: sapien.Pose,
        radius,
        color=None,
        density=1000.0,
        physical_material: sapien.PhysicalMaterial = None,
        name='',
) -> sapien.Actor:
    """Create a sphere."""
    builder = scene.create_actor_builder()
    builder.add_sphere_collision(radius=radius, material=physical_material, density=density)
    builder.add_sphere_visual(radius=radius, color=color)
    sphere = builder.build(name=name)
    sphere.set_pose(pose)
    return sphere

> Note: The rolling resistance (friction) is not modeled in SAPIEN currently.

## Create a kinematic actor

Now, let’s create a slope. The slope should be a **kinematic** object, rather than a **dynamic** object. In other words, it can not be affected by external forces. We can set `is_kinematic=True` when building the actor.

In [ ]:
def create_box(
        scene: sapien.Scene,
        pose: sapien.Pose,
        half_size,
        color=None,
        is_kinematic=False,
        density=1000.0,
        physical_material: sapien.PhysicalMaterial = None,
        name='',
) -> sapien.Actor:
    """Create a box.

    Args:
        scene: sapien.Scene to create a box.
        pose: 6D pose of the box.
        half_size: [3], half size along x, y, z axes.
        color: [3] or [4], rgb or rgba.
        is_kinematic: whether an object is kinematic (can not be affected by forces).
        density: float, the density of the box.
        physical_material: physical material of the actor.
        name: name of the actor.

    Returns:
        sapien.Actor
    """
    half_size = np.array(half_size)
    builder = scene.create_actor_builder()
    builder.add_box_collision(half_size=half_size, material=physical_material, density=density)  # Add collision shape
    builder.add_box_visual(half_size=half_size, color=color)  # Add visual shape
    if is_kinematic:
        box = builder.build_kinematic(name=name)
    else:
        box = builder.build(name=name)
    box.set_pose(pose)
    return box

The following provided code will build the rest of the scene.

In [ ]:
scene.add_ground(altitude=0, material=physical_material)

# Slope
half_size = [0.25, 0.5, 0.05]
z_offset = offset
angle = np.deg2rad(angle)
slope_pose = sapien.Pose(
    p=[0, 0, half_size[1] * np.sin(angle) + half_size[2] * np.cos(angle) + z_offset],
    q=axangle2quat([1.0, 0.0, 0.0], angle),
)
slope = create_box(
    scene,
    slope_pose,
    half_size=half_size,
    color=[0.5, 0.5, 0.5],
    name='slope',
    is_kinematic=True,
    physical_material=physical_material
)

if obj == 'box':
    box_half_size = 0.05
    box_pose = sapien.Pose(
        p=[0,
            (half_size[1] - box_half_size) * np.cos(angle) -
            (half_size[2] + box_half_size) * np.sin(angle),
            (half_size[1] - box_half_size) * np.sin(angle) +
            (half_size[2] + box_half_size) * np.cos(angle) + slope_pose.p[2]],
        q=axangle2quat([1.0, 0.0, 0.0], angle),
    )
    actor = create_box(
        scene,
        box_pose,
        half_size=[box_half_size] * 3,
        color=[0., 0., 1.],
        physical_material=physical_material,
        name='box',
    )
elif obj == 'sphere':
    # NOTE: Since Sapien does not model rolling resistance (friction), the sphere will roll forever.
    # However, you can set actor's damping, like air resistance.
    radius = 0.05
    sphere_pose = sapien.Pose(
        p=[0,
            (half_size[1] - radius) * np.cos(angle) -
            (half_size[2] + radius) * np.sin(angle),
            (half_size[1] - radius) * np.sin(angle) +
            (half_size[2] + radius) * np.cos(angle) + slope_pose.p[2]],
    )
    actor = create_sphere(
        scene,
        sphere_pose,
        radius=0.05,
        color=[0., 1., 0.],
        physical_material=physical_material,
        name='sphere',
    )
else:
    raise NotImplementedError()

## Set damping for the actor

Sometimes, you might model some resistance proportional to (linear or angular) velocity, like air resistance. It can be achieved by setting the **damping** of an actor.

In [ ]:
actor.set_damping(linear=linear_damping, angular=angular_damping)

## Get kinematic quantities (pose, velocity) of an actor

We can acquire kinematic quantities (pose, linear velocity, angular velocity) of an actor through `get_pose()`, `get_velocity()`, `get_angular_velocity()`. Let's see the kinematic properties together with visualizations during the object sliding down the slope.

In [ ]:
scene.set_ambient_light([0.5, 0.5, 0.5])
scene.add_directional_light([0, 1, -1], [0.5, 0.5, 0.5])

camera = scene.add_camera(name='camera', width=1024, height=768, fovy=1.57, near=0.1, far=100)
camera.set_pose(sapien.Pose([-1.04541, 0, 1.54541], [0.92388, 0, 0.382683, 0]))

plt.figure(figsize=(15, 6))

for steps in range(100):
    scene.step()

    if steps % 10 == 0:
        # Visualization
        scene.update_render()
        camera.take_picture()
        rgba = camera.get_color_rgba()
        plt.subplot(2, 5, steps//10+1)
        plt.imshow(rgba[..., :3])

        # Get kinematic quantities
        print('step:', steps)
        print('Pose', actor.get_pose())
        print('Velocity', actor.get_velocity())
        print('Angular velocity', actor.get_angular_velocity())

plt.show()

You can restart the runtime, adjust the hyperparameters (at the beginning of the notebook) and rerun the above cells to see how changing physical properties will affect the motion of the object.

> Note: If you are working locally, you can use `Viewer` to see the complete animation of the motion. For more information please refer to the comprehensive tutorial linked at the top.